In [22]:
import scipy as sp
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import sys
import os
import random
from timeit import default_timer as timer
train = pd.read_csv('trainingSet.csv')
test = pd.read_csv('testSet.csv')
train = train.sample(random_state=18,frac=1)
train_split = train.sample(random_state=32 ,frac=0.5)


In [26]:
S1,S2,S3,S4,S5,S6,S7,S8,S9,S10 =\
train_split[:260],train_split[260:520],train_split[520:780],train_split[780:1040],\
train_split[1040:1300],train_split[1300:1560],train_split[1560:1820],\
train_split[1820:2080],train_split[2080:2340],train_split[2340:2600]
sets = [[]]
depths = [3,5,7,9]
idx_sets = [S1,S2,S3,S4,S5,S6,S7,S8,S9,S10]
#idx_sets = [S1,S2,S3]

In [27]:
def gini_gain(df):
    dict_gini={}
    features = list(df.keys()[:-1]) 
    valcounts= df.apply(pd.Series.value_counts)/len(df)
    main_gini = 1- (((float(df.query('decision == 0').decision.count()))/float(len(df['decision'])))**2+ \
                     (float(df.query('decision == 1').decision.count())/float(len(df['decision'])))**2)
    values = [0,1]
    valcounts= df.apply(pd.Series.value_counts)/len(df)
    for feature in features:
        df_feaval0 = df[df[feature]==0]
        df_feaval1 = df[df[feature]==1]
        if(len(df_feaval0) ==0):
            gini_feature0 = 0
            #continue
        else:
            gini_feature0 = 1- (((float(df_feaval0.query('decision == 0').decision.count()))/float(len(df_feaval0)))**2+ \
                         (float(df_feaval0.query('decision == 1').decision.count())/float(len(df_feaval0)))**2)
        if(len(df_feaval1) ==0):
            gini_feature1 = 0
        else:
            gini_feature1 = 1- (((float(df_feaval1.query('decision == 0').decision.count()))/float(len(df_feaval1)))**2+ \
                         (float(df_feaval1.query('decision == 1').decision.count())/float(len(df_feaval1)))**2)
        fea_gain = main_gini - (valcounts[feature][0]*gini_feature0 + valcounts[feature][1]*gini_feature1)
        dict_gini[feature] = fea_gain
    root = max(dict_gini, key=dict_gini.get)
    ginival = dict_gini[root]
    return root,ginival
def get_split(df):
    root,ginival = gini_gain(df)
    lchild, rchild = df[df[root]==0],df[df[root]==1]
    return {'lchild':lchild, 'rchild':rchild,'gini':ginival,'root':root}
def topvalues(df):
    yes = float(df.query('decision == 1').decision.count())
    no = float(df.query('decision == 0').decision.count())
    if max(yes,no) == yes:
        return 1
    else:
        return 0
def getfinal_pred(dict_pred,num_of_trees):
    predict = 0
    df = pd.DataFrame().astype(float)
    for i in range(num_of_trees):
        df[str(i)+ "_predict"] = dict_pred[i]
    predict = df.mode(axis=1)
    return predict[0]

def build_tree(df,max_depth,min_elements,curr_depth):
    print max_depth
    treedata = get_split(df)
    if curr_depth < max_depth: 
        if len(treedata['lchild']) > min_elements:
            remtree = treedata['lchild']
            remtree = remtree.drop(treedata['root'], 1)
            treedata['lchild'] = build_tree(remtree,max_depth,min_elements,curr_depth+1)
        else:
            treedata['lchild'] = topvalues(treedata['lchild'])
        if len(treedata['rchild']) > min_elements:
            remtree1 = treedata['rchild']
            remtree1 = remtree1.drop(treedata['root'], 1)
            treedata['rchild'] = build_tree(remtree1,max_depth,min_elements,curr_depth+1)
        else:
            treedata['rchild'] = topvalues(treedata['rchild'])
    else:
        treedata['lchild'] = topvalues(treedata['lchild'])
        treedata['rchild'] = topvalues(treedata['rchild'])
    return treedata
def traverse_dict(instance,tree_dup,predict): 
    if (instance[tree_dup[tree_dup.keys()[3]]]) == 0:
        if type(tree_dup[tree_dup.keys()[0]]) is dict:
            predict = traverse_dict(instance,tree_dup[tree_dup.keys()[0]],predict)
        elif type(tree_dup[tree_dup.keys()[0]]) is not dict:
            #predict = tree_dup[tree_dup.keys()[0]]
            predict =  tree_dup['lchild']
            return predict
    elif (instance[tree_dup[tree_dup.keys()[3]]]) == 1:
        if type(tree_dup[tree_dup.keys()[1]]) is dict:
            predict = traverse_dict(instance,tree_dup[tree_dup.keys()[1]],predict)
        elif type(tree_dup[tree_dup.keys()[1]]) is not dict:
            predict =  tree_dup['rchild']
           # print tree_dup
            return predict
    #print predict
    return predict
def bagging(train, test,max_depth):
    print "BT", max_depth
    tree_bag ={}
    train_pred={}
    test_pred ={}
    predict_train = 0
    predict_test = 0
    training_final = train
    test_final = test
    num_of_trees = 30
    for i in range(num_of_trees):
        print "BT",i
        train_bag = train.sample(frac = 1.0, replace = True)
        train_bag.reset_index(drop = True)
        tree = build_tree(train_bag,max_depth,50,1)
        #train_pred[i] = train.apply(predict_dec,axis=1,args = (tree,))
        test_pred[i] = test.apply(predict_dec,axis=1,args = (tree,))
    return test_pred
def gini_gain_RF(df):
    dict_gini={}
    full_features = list(df.keys()[:-1]) 
    choices = random.sample(range(len(full_features)), int(np.sqrt(len(df.keys()))))
    features = [full_features[i] for i in choices]
    valcounts= df.apply(pd.Series.value_counts)/len(df)
    main_gini = 1- (((float(df.query('decision == 0').decision.count()))/float(len(df['decision'])))**2+ \
                     (float(df.query('decision == 1').decision.count())/float(len(df['decision'])))**2)
    values = [0,1]
    valcounts= df.apply(pd.Series.value_counts)/len(df)
    for feature in features:
        df_feaval0 = df[df[feature]==0]
        df_feaval1 = df[df[feature]==1]
        if(len(df_feaval0) ==0):
            #gini_feature0 = 1
            continue
        else:
            gini_feature0 = 1- (((float(df_feaval0.query('decision == 0').decision.count()))/float(len(df_feaval0)))**2+ \
                         (float(df_feaval0.query('decision == 1').decision.count())/float(len(df_feaval0)))**2)
        if(len(df_feaval1) ==0):
            continue
            #gini_feature1 = 1
        else:
            gini_feature1 = 1- (((float(df_feaval1.query('decision == 0').decision.count()))/float(len(df_feaval1)))**2+ \
                         (float(df_feaval1.query('decision == 1').decision.count())/float(len(df_feaval1)))**2)
        fea_gain = main_gini - (valcounts[feature][0]*gini_feature0 + valcounts[feature][1]*gini_feature1)
        dict_gini[feature] = fea_gain
    root = max(dict_gini, key=dict_gini.get)
    ginival = dict_gini[root]
    return root,ginival
def get_split_RF(df):
    root,ginival = gini_gain_RF(df)
    lchild, rchild = df[df[root]==0],df[df[root]==1]
    return {'lchild':lchild, 'rchild':rchild,'gini':ginival,'root':root}

def build_tree_RF(df,max_depth,min_elements,curr_depth):
    treedata = get_split_RF(df)
    if curr_depth <max_depth : 
        if len(treedata['lchild']) > min_elements:
            remtree = treedata['lchild']
            remtree = remtree.drop(treedata['root'], 1)
            treedata['lchild'] = build_tree_RF(remtree,max_depth,min_elements,curr_depth+1)
        else:
            treedata['lchild'] = topvalues(treedata['lchild'])
        if len(treedata['rchild']) > min_elements:
            remtree1 = treedata['rchild']
            remtree1 = remtree1.drop(treedata['root'], 1)
            treedata['rchild'] = build_tree_RF(remtree1,max_depth,min_elements,curr_depth+1)
        else:
            treedata['rchild'] = topvalues(treedata['rchild'])
    else:
        treedata['lchild'] = topvalues(treedata['lchild'])
        treedata['rchild'] = topvalues(treedata['rchild'])
    return treedata
def randomforest(train, test,max_depth):
    print "RF",max_depth
    tree_bag ={}
    train_pred={}
    test_pred ={}
    predict_train = 0
    predict_test = 0
    training_final = train
    test_final = test
    num_of_trees = 30
    for i in range(num_of_trees):
        print "RF", i
        train_bag = train.sample(frac = 1.0, replace = True)
        train_bag.reset_index(drop = True)
        tree = build_tree_RF(train_bag,max_depth,50,1)
        #train_pred[i] = train.apply(predict_dec,axis=1,args = (tree,))
        test_pred[i] = test.apply(predict_dec,axis=1,args = (tree,))
    return test_pred

def predict_dec(df,tree_dup):
    predict = 0
    predict = traverse_dict(df,tree_dup,0)
    return predict
def printaccuracy(decision,predict):
    accuracy = abs(decision-predict)
    accuracy =  round(float((accuracy==0).sum())/((accuracy==0).sum()+(accuracy==1).sum()),2)
    return accuracy   

In [28]:
accuracy_DT={}
standard_error_DT =[]
accuracy_BT ={}
standard_error_BT =[]
accuracy_RF ={}
standard_error_RF = []
for depth in depths:
    print depth
    idx_itr =0
    cross_accuracy_DT=[]
    cross_accuracy_BT =[]
    cross_accuracy_RF=[]
    for set in idx_sets:
        test = set
        if(idx_itr>0 and idx_itr<9):
            set1 = pd.concat(idx_sets[0:idx_itr],ignore_index=True)
            set2 = pd.concat(idx_sets[idx_itr+1:],ignore_index=True)
            S_C = pd.concat([set1, set2],ignore_index=True)
        elif(idx_itr==0):
            set2 = pd.concat(idx_sets[idx_itr+1:],ignore_index=True)
            S_C = set2
        elif(idx_itr==len(idx_sets)-1):
            set1 = pd.concat(idx_sets[0:idx_itr],ignore_index=True)
            S_C = set1
        else:
            print "Debug your code!"
        idx_itr +=1
        train = S_C
        #tree = build_tree(train,depth,50,1)
        #predict_DT = test.apply(predict_dec,axis=1,args = (tree,))
        #test_accuracy_DT = printaccuracy(test['decision'],predict_DT)
        #cross_accuracy_DT.append(test_accuracy_DT)
        train_pred_BT,test_pred_BT = bagging(train, test,depth)
        predict_test_BT = getfinal_pred(test_pred_BT,30)
        test_accuracy_BT = printaccuracy(test['decision'],predict_test_BT)
        cross_accuracy_BT.append(test_accuracy_BT)
        train_pred_RF,test_pred_RF = randomforest(train, test,depth)
        predict_test_RF = getfinal_pred(test_pred_RF,30)
        test_accuracy_RF = printaccuracy(test['decision'],predict_test_RF)
        cross_accuracy_RF.append(test_accuracy_RF)
        
    #accuracy_DT[depth] = round(np.float(np.mean(cross_accuracy_DT)),2)
    #standard_error_DT.append(round(np.std(cross_accuracy_DT,dtype=np.float32)/np.sqrt(len(cross_accuracy_DT)),3))
    accuracy_BT[depth] = round(np.float(np.mean(cross_accuracy_BT)),2)
    standard_error_BT.append(round(np.std(cross_accuracy_BT,dtype=np.float32)/np.sqrt(len(cross_accuracy_BT)),3))
    accuracy_RF[depth] = round(np.float(np.mean(cross_accuracy_RF)),2)
    standard_error_RF.append(round(np.std(cross_accuracy_RF,dtype=np.float32)/np.sqrt(len(cross_accuracy_RF)),3))
print "DT accuracies: ", accuracy_DT
print "DT standard errors",standard_error_DT
print "BT accuracies: ", accuracy_BT
print "BT standard errors",standard_error_BT
print "RF accuracies: ", accuracy_RF
print "RF standard errors",standard_error_RF

3
BT 3
BT 0
3
3
3
3
3
3
3
BT 1
3
3
3
3
3
3
3
BT 2
3
3
3
3
3
3
3
BT 3
3
3
3
3
3
3
3
BT 4
3
3
3
3
3
3
3
BT 5
3
3
3
3
3
3
3
BT 6
3
3
3
3
3
3
3
BT 7
3
3
3
3
3
3
3
BT 8
3
3
3
3
3
3
3
BT 9
3
3
3
3
3
3
3
BT 10
3
3
3
3
3
3
3
BT 11
3
3
3
3
3
3
3
BT 12
3
3
3
3
3
3
3
BT 13
3
3
3
3
3
3
3
BT 14
3
3
3
3
3
3
3
BT 15
3
3
3
3
3
3
3
BT 16
3
3
3
3
3
3
3
BT 17
3
3
3
3
3
3
3
BT 18
3
3
3
3
3
3
3
BT 19
3
3
3
3
3
3
3
BT 20
3
3
3
3
3
3
3
BT 21
3
3
3
3
3
3
3
BT 22
3
3
3
3
3
3
3
BT 23
3
3
3
3
3
3
3
BT 24
3
3
3
3
3
3
3
BT 25
3
3
3
3
3
3
3
BT 26
3
3
3
3
3
3
3
BT 27
3
3
3
3
3
3
3
BT 28
3
3
3
3
3
3
3
BT 29
3
3
3
3
3
3
3
RF 3
RF 0
RF 1
RF 2
RF 3
RF 4
RF 5
RF 6
RF 7
RF 8
RF 9
RF 10
RF 11
RF 12
RF 13
RF 14
RF 15
RF 16
RF 17
RF 18
RF 19
RF 20
RF 21
RF 22
RF 23
RF 24
RF 25
RF 26
RF 27
RF 28
RF 29
BT 3
BT 0
3
3
3
3
3
3
3
BT 1
3
3
3
3
3
3
3
BT 2
3
3
3
3
3
3
3
BT 3
3
3
3
3
3
3
BT 4
3
3
3
3
3
3
3
BT 5
3
3
3
3
3
3
3
BT 6
3
3
3
3
3
3
3
BT 7
3
3
3
3
3
3
3
BT 8
3
3
3
3
3
3
BT 9
3
3
3
3
3
3
3
BT 10
3
3
3
3
3
3
3
BT 11
3
3
3
3
3
3

5
5
5
5
5
5
5
5
BT 11
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
BT 12
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
BT 13
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
BT 14
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
BT 15
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
BT 16
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
BT 17
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
BT 18
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
BT 19
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
BT 20
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
BT 21
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
BT 22
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
BT 23
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
BT 24
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
BT 25
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
BT 26
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
BT 27
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
BT 28
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
BT 29
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
RF

5
BT 3
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
BT 4
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
BT 5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
BT 6
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
BT 7
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
BT 8
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
BT 9
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
BT 10
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
BT 11
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
BT 12
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
BT 13
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
BT 14
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
BT 15
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
BT 16
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
BT 17
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
BT 18
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
BT 19
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
BT 20
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
BT 21
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
BT 22
5
5
5
5
5
5
5
5
5
5

RF 1
RF 2
RF 3
RF 4
RF 5
RF 6
RF 7
RF 8
RF 9
RF 10
RF 11
RF 12
RF 13
RF 14
RF 15
RF 16
RF 17
RF 18
RF 19
RF 20
RF 21
RF 22
RF 23
RF 24
RF 25
RF 26
RF 27
RF 28
RF 29
7
BT 7
BT 0
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
BT 1
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
BT 2
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
BT 3
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
BT 4
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
BT 5
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
BT 6
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
BT 7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
BT 8
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7

7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
BT 18
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
BT 19
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
BT 20
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
BT 21
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
BT 22
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
BT 23
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
BT 24
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
BT 25
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
BT 26
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
BT 27
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
BT 28
7
7


...

9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
BT 4
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
BT 5
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
BT 6
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
BT 7
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
BT 8
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
BT 9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
BT 10
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9


9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
BT 29
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
RF 9
RF 0
RF 1
RF 2
RF 3
RF 4
RF 5
RF 6
RF 7
RF 8
RF 9
RF 10
RF 11
RF 12
RF 13
RF 14
RF 15
RF 16
RF 17
RF 18
RF 19
RF 20
RF 21
RF 22
RF 23
RF 24
RF 25
RF 26
RF 27
RF 28
RF 29
BT 9
BT 0
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
BT 1
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
BT 2
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
BT 3
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
BT

9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
BT 24
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
BT 25
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
BT 26
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
BT 27
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
BT 28
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
BT 29
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
RF 9
RF 0
RF 1
RF 2
RF 3
RF 4
RF 5
RF 6
RF 7
RF 8
RF 9
RF 10
RF 11

9
9
9
9
9
9
9
9
9
BT 19
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
BT 20
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
BT 21
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
BT 22
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
BT 23
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
BT 24
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
BT 25
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
BT 26
9
9
9
9
9
9
9
9
9
9


9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
BT 15
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
BT 16
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
BT 17
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
BT 18
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
BT 19
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
BT 20
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
BT 21
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
